In [12]:
import time
from copy import deepcopy
from hashlib import sha256

## Hash

Create hash digest from string, int or float.

In [13]:
sha = lambda x: sha256(str(x).encode()).hexdigest(); sha('satoshi')

'da2876b3eb31edb4436fa4650673fc6f01f90de2f1793c4ec332b2387b09726f'

Create random hash for testing.

In [14]:
rh = lambda: sha(time.time())
h  = rh(); h

'4944978b1be90f80197ae690deeaba2ebc7d32f57b963a4c97289164b301a10f'

Turn hash into one of 256 possible emojis. 

In [15]:
def hash2emoji(h):
    if h[:2]!='0x': h='0x'+h
    offset  = int(h[0:4], 0)
    unicode = b'\U' + b'000' + str(hex(0x1F466+offset))[2:].encode()
    return unicode.decode('unicode_escape')

hash2emoji(h)

'💯'

Print hash in a nice way.

In [16]:
def ph(h):
    if len(h)<16: return h
    else        : return hash2emoji(h)+' '+h[:16] + '...' + h[-3:]

ph(h)

'💯 4944978b1be90f80...10f'

## Transaction

A transaction (abbr. tx) is used for transfering `value` from one address `fr` to another `to`.

The `nonce` is the number of transactions sent from a given address. It's used to avoid replay attacks. For a more detailed explanation: https://kb.myetherwallet.com/en/transactions/what-is-nonce/

In [17]:
class TX: 
    def __init__(self, fr, to, value, nonce): 
        self.fr, self.to = fr, to
        self.value       = value
        self.nonce       = nonce
        self.time        = time.time()
        
    def __setattr__(self, prop, val):
        super().__setattr__(prop, val)
        
        # recalculate hash every time if it is not the signature itself
        if prop!='sig': super().__setattr__('hash', sha(self.__dict__))
        
    def __str__ (self): 
        is_signed = '\nsigned:\ttrue' if hasattr(self, 'sig') else '\nsigned:\tfalse'
        return ('time:\t'   + time.ctime(self.time)+
                                '\nfrom:\t' +ph(self.fr)+
                                '\nto:\t'   +ph(self.to)+
                                '\nvalue:\t'+ str(self.value)+' ether' 
                                '\nnonce:\t'+ str(self.nonce)+
                                '\nhash:\t' + ph(self.hash)+
                                 is_signed)
    def __bytes__(self): return str(self.hash).encode()
    def __eq__(self, other): return self.hash == other.hash

In [18]:
tx1 = TX(rh(), rh(), 12, 0)
tx2 = TX(rh(), rh(), 12, 0)

In [19]:
def txs2str(txs): return '\n'.join([str(tx)+'\n' for tx in txs])

print(txs2str([tx1,tx2]))

time:	Fri Mar 26 19:43:33 2021
from:	📮 886cc8133b71546f...460
to:	📸 9255401c7da6c138...8f6
value:	12 ether
nonce:	0
hash:	💣 3d02226b061e8acd...117
signed:	false

time:	Fri Mar 26 19:43:33 2021
from:	🔮 c8b3628bd1835bf8...11e
to:	📼 968bab2e79a8abd4...50a
value:	12 ether
nonce:	0
hash:	💯 490890d4cf87806c...d0b
signed:	false



#### Signing Tx
An account (implemented in `02_account.ipynb`) has the ability to sign a tx. Here we simply mock a signature with a random hash.

In [20]:
tx1.sig = rh(); print(tx1)

time:	Fri Mar 26 19:43:33 2021
from:	📮 886cc8133b71546f...460
to:	📸 9255401c7da6c138...8f6
value:	12 ether
nonce:	0
hash:	💣 3d02226b061e8acd...117
signed:	true


#### Changing Tx
Every change in the object is reflected by its hash. Compare the signed tx hash below with the hash of the unsigned tx above. The hash has changed and the signed flag is true. 

In [21]:
tx2_false_value       = deepcopy(tx2)
tx2_false_value.value = 120
print(tx2)

time:	Fri Mar 26 19:43:33 2021
from:	🔮 c8b3628bd1835bf8...11e
to:	📼 968bab2e79a8abd4...50a
value:	12 ether
nonce:	0
hash:	💯 490890d4cf87806c...d0b
signed:	false


Transactions can be determined unequal by simply comparing the hashes as implemented in `__eq__`.

In [22]:
assert not tx2 == tx2_false_value